In [ ]:
!pip install flask_ngrok
!pip install pyngrok
!pip install flask-wtf
!pip install flask-sqlalchemy
!ngrok authtoken 36L7OIVigcPBvxV4Cr4Rqo9F89v_7iM67rTKDJLtQ4ZkNvncK

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.5/152.5 kB 3.8 MB/s eta 0:00:00
Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
from google.colab import drive
import os
from flask_ngrok import run_with_ngrok
from flask import Flask, request, render_template, redirect, url_for
from pyngrok import ngrok
from flask_sqlalchemy import SQLAlchemy
drive.mount('/content/drive')
PROJECT_ROOT = '/content/drive/MyDrive/colab/flask_colab'
DB_PATH = os.path.join(PROJECT_ROOT, 'database.db')
app = Flask(
    __name__,
    template_folder=os.path.join(PROJECT_ROOT, 'templates'),
    static_folder=os.path.join(PROJECT_ROOT, 'static')
)

app.config['SQLALCHEMY_DATABASE_URI'] = f"sqlite:///{DB_PATH}"
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False
db = SQLAlchemy(app)
class Hotel(db.Model):
    __tablename__ = 'hotels'
    id = db.Column(db.Integer, primary_key=True)
    code = db.Column(db.String(20), nullable=False)
    name = db.Column(db.String(100), nullable=False)
    rooms = db.Column(db.Integer, nullable=False)
    price = db.Column(db.Float, nullable=False)
    def __repr__(self):
        return f"<Hotel {self.name}>"
with app.app_context():
    db.create_all()
    if Hotel.query.count() == 0:
        h1 = Hotel(code="KS01", name="Khách Sạn Hoa Mai", rooms=20, price=500000)
        h2 = Hotel(code="KS02", name="Khách Sạn Biển Xanh", rooms=15, price=750000)
        db.session.add_all([h1, h2])
        db.session.commit()
try:
    public_url = ngrok.connect(5000)
    print("Flask Public URL:", public_url)
except:
    print("Ngrok lỗi, vẫn chạy local")
@app.route("/")
def home():
    return render_template("index.html")
@app.route("/about")
def about():
    return render_template("about.html")
@app.route("/contact")
def contact():
    return render_template("contact.html")
@app.route("/login", methods=['GET','POST'])
def login():
    if request.method == "POST":
        username = request.form["username"]
        password = request.form["password"]

        if username.lower() == "vietanh" and password == "123456":
            return redirect(url_for("about"))
        else:
            return render_template("login.html", error="Sai tài khoản hoặc mật khẩu!")

    return render_template("login.html")
@app.route("/hotel")
def hotel_list():
    keyword = request.args.get("keyword", "")
    if keyword:
        hotels = Hotel.query.filter(Hotel.name.contains(keyword)).all()
    else:
        hotels = Hotel.query.all()
    return render_template("hotel_list.html", hotels=hotels, keyword=keyword)

@app.route("/hotel/add", methods=['GET','POST'])
def hotel_add():
    if request.method == "POST":
        code = request.form["code"]
        name = request.form["name"]
        rooms = int(request.form["rooms"])
        price = float(request.form["price"])

        new_hotel = Hotel(code=code, name=name, rooms=rooms, price=price)
        db.session.add(new_hotel)
        db.session.commit()
        return redirect(url_for("hotel_list"))
    return render_template("hotel_add.html")
@app.route("/hotel/edit/<int:id>", methods=['GET','POST'])
def hotel_edit(id):
    hotel = Hotel.query.get_or_404(id)
    if request.method == "POST":
        hotel.code = request.form["code"]
        hotel.name = request.form["name"]
        hotel.rooms = int(request.form["rooms"])
        hotel.price = float(request.form["price"])
        db.session.commit()
        return redirect(url_for("hotel_list"))
    return render_template("hotel_edit.html", hotel=hotel)
@app.route("/hotel/delete/<int:id>")
def hotel_delete(id):
    hotel = Hotel.query.get_or_404(id)
    db.session.delete(hotel)
    db.session.commit()
    return redirect(url_for("hotel_list"))
if __name__ == "__main__":
    app.run()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Flask Public URL: NgrokTunnel: "https://chaste-mina-inanimately.ngrok-free.dev" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [04/Dec/2025 04:29:18] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [04/Dec/2025 04:29:31] "GET /login HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [04/Dec/2025 04:29:32] "GET /static/images/logo1.png HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [04/Dec/2025 04:29:43] "POST /login HTTP/1.1" 302 -
INFO:werkzeug:127.0.0.1 - - [04/Dec/2025 04:29:43] "GET /about HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [04/Dec/2025 05:24:09] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [04/Dec/2025 05:24:58] "GET /contact HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [04/Dec/2025 05:24:59] "GET /static/logo.png HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [04/Dec/2025 05:25:03] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [04/Dec/2025 05:25:05] "GET /hotel HTTP/1